## Stage 1

In [ ]:
import pandas as pd

In [ ]:
def insert_values(df_dashboard, df_values, col_join, col_values): # df_values should have "values" column
    df_dashboard.loc[df_dashboard[col_join].isin(df_values[col_join]), col_values] = df_values.loc[df_values[col_join].isin(df_dashboard[col_join]), 'values'].values
    df_dashboard[col_values] = df_dashboard[col_values].fillna(0).astype(int)
    return df_dashboard[col_values]

In [ ]:
# Define names and folders
relative_folder = "../data/"

programs_file = "programs.xlsx"

bitrix_file = "bitrix.xls"

master_file = "asav.xlsx"

template_file = "template.xlsx"

In [ ]:
col_applications = """Кол-во регистраций в ЛК абитуриента
(РФ 1 и 2 приоритет)"""
col_contracts = """Договоры
(ПК) РФ"""
col_payments = """Оплаты
(ПК)"""
col_enrollments = "Зачисленные (ПК)"

In [ ]:
# cчитываем базу данных програм
df_online_programs = pd.read_excel(relative_folder + programs_file)
df_online_programs = df_online_programs[df_online_programs['format'] != 'offline'].reset_index(drop=True)
df_online_programs[['plan_rus', 'plan_foreign']] = df_online_programs[['plan_rus', 'plan_foreign']].astype(int)
df_online_master_programs = df_online_programs[df_online_programs['level'] == 'master'].drop(columns=["format"]).sort_values(by="program").reset_index(drop=True)
df_online_bachelor_programs = df_online_programs[df_online_programs['level'] == 'bachelor'].sort_values(by="program").reset_index(drop=True)


In [ ]:
%%time
df_master = pd.read_excel(relative_folder + master_file) #, skiprows=1)


In [ ]:
%%time
df_master = pd.read_excel(relative_folder + master_file, sheet_name="только онлайн", skiprows=1, usecols="L:DT")


In [ ]:

master_file_sheet_name = "только онлайн"
master_col_programs = "Конкурс в магистратуру"
master_col_contracts = "Договор на обучение"
master_col_payments = "Оплата первого периода" # "Оплачено"
master_col_enrollments = "Приказ о зачислении"

# достаем данные по ЛК, договорам, оплатам и зачислениям из АСАВ
master_applications = df_master.groupby(master_col_programs)[master_col_programs].count() #.rename("program")#.sort_values(ascending=False)
master_applications = pd.DataFrame({'program':master_applications.index, 'values':master_applications.values})

master_contracts = df_master[df_master[master_col_contracts].notna()].groupby(master_col_programs)[master_col_programs].count()
master_contracts = pd.DataFrame({'program':master_contracts.index, 'values':master_contracts.values})

master_payments = df_master[df_master[master_col_payments] == "Оплачено"].groupby(master_col_programs)[master_col_programs].count()
master_payments = pd.DataFrame({'program':master_payments.index, 'values':master_payments.values})

master_enrollments = df_master[df_master[master_col_enrollments].notna()].groupby(master_col_programs)[master_col_programs].count()
master_enrollments = pd.DataFrame({'program':master_enrollments.index, 'values':master_enrollments.values})


In [ ]:
# создаем дашборд по магистратурам добавляя туда программы из базы
df_master_dashboard = pd.read_excel(relative_folder + template_file)
df_master_dashboard = pd.concat([df_online_master_programs, df_master_dashboard])
.fillna(0)
df_master_dashboard[['plan_rus', 'plan_foreign']] = df_master_dashboard[['plan_rus', 'plan_foreign']].astype(int)


In [ ]:
# вставляем в дашборд показатели из АСАВ

df_master_dashboard[col_applications] = insert_values(df_master_dashboard, master_applications, 'program', col_applications)
df_master_dashboard[col_contracts] = insert_values(df_master_dashboard, master_contracts, 'program', col_contracts)
df_master_dashboard[col_payments] = insert_values(df_master_dashboard, master_payments, 'program', col_payments)
df_master_dashboard[col_enrollments] = insert_values(df_master_dashboard, master_enrollments, 'program', col_enrollments)

# df_master_dashboard.loc[df_master_dashboard['program'].isin(master_applications['program']), col_applications] = master_applications.loc[master_applications['program'].isin(df_master_dashboard['program']), 'applications'].values
# df_master_dashboard[col_applications] = df_master_dashboard[col_applications].fillna(0).astype(int)
# df_master_dashboard.loc[:, [col_applications]] = master_applications
# df_master_dashboard.join(master_applications, on="program", how="left", lsuffix=col_applications)

In [ ]:
df_master_dashboard[['program', col_applications, col_contracts, col_payments, col_enrollments]]

In [ ]:
df_bitrix = pd.read_html(relative_folder + bitrix_file, header=0)[0]


In [ ]:
main_studyonline = "Общий лендинг"
try:
    df_bitrix = pd.read_html(relative_folder + bitrix_file, header=0)[0]
    df_bitrix['Образовательная программа'].fillna(main_studyonline, inplace=True)

    # pd.read_excel(relative_folder + bitrix_file)
except:
    print("Нет выгрузки из Битрикса или она называется не " + bitrix_file)
    df_bitrix = pd.DataFrame()

leads = df_bitrix.groupby('Образовательная программа')['Образовательная программа'].count()
leads = pd.DataFrame({'program_bitrix':leads.index, 'values':leads.values})
# leads['program'] = insert_values()

main_leads = leads.loc[leads['program_bitrix'] == main_studyonline, 'values'].values[0]

## Stage 2

In [ ]:
#%pip install jupyter-dash dash-express

In [1]:
def insert_values(df_dashboard, df_values, col_join, col_values): # df_values should have "values" column
    df_dashboard.loc[df_dashboard[col_join].isin(df_values[col_join]), col_values] = df_values.loc[df_values[col_join].isin(df_dashboard[col_join]), 'values'].values
    df_dashboard[col_values] = df_dashboard[col_values].fillna(0).astype(int)
    return df_dashboard[col_values]

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
# папки и файлы для загрузки
relative_folder = "../data/"

programs_file = "programs.xlsx"

dashboard_file = "dashboard.xlsx"

bitrix_file = "bitrix.xls"

master_file = "asav.xlsx"
master_file_sheet_name = "только онлайн"
master_col_programs = "Конкурс в магистратуру"
master_col_contracts = "Договор на обучение" # не пусто
master_col_payments = "Оплата первого периода" # "Оплачено"
master_col_enrollments = "Приказ о зачислении" # не пусто

bachelor_app_file = "bac_applications.xls"

bachelor_con_file = "bac_contracts.xls"

bachelor_enr_file = "bac_enrolled.xlsx"

main_studyonline = "Общий лендинг"

template_file = "template.xlsx"

# основные параметры
col_program = 'program'
col_plan = 'plan_rus'
col_leads = "Общее кол-во заявок (studyonline) ВСЕГО"
col_applications = """Кол-во регистраций в ЛК абитуриента (РФ 1 и 2 приоритет)"""
col_contracts = """Договоры (ПК) РФ"""
col_payments = """Оплаты (ПК)"""
col_enrollments = "Зачисленные (ПК)"

# дополнительные и расчетные параметры
col_leads_total = """Общее кол-во заявок ВСЕГО (портал+РК)"""
col_leads_partners = """Общее кол-во заявок  c  hse.ru по кнопкам/партнерских стр"""
col_conversion_leads_to_contracts = """Конверсия заявка -> договор (без учета заявок с общего ленда)"""
col_needed_applications = """Необходимо регистраций в ЛК для обеспечения набора (30% поступили от регистрации в АСАВ)"""
NEEDED_APPLICATIONS_RATIO = 30 / 100 #percents
col_conversion_applications_to_contracts = """Конверсия ЛК -> договор"""
col_conversion_contracts_to_payments = """Конверсия договор -> оплата"""
col_conversion_contracts_to_enrollments = """Конверсия договор -> зачисление"""
col_payments_div_plan = """Выполнение плана %"""


In [ ]:
# считывание файлов

try:
    # cчитываем базу данных програм
    print("Начинаем считывать базу программ")
    df_online_programs = pd.read_excel(relative_folder + programs_file)
    df_online_programs = df_online_programs[df_online_programs['format'] != 'offline'].reset_index(drop=True)
    #df_online_programs[['plan_rus', 'plan_foreign']] = df_online_programs[['plan_rus', 'plan_foreign']].astype(int)
    df_online_master_programs = df_online_programs[df_online_programs['level'] == 'master'].drop(columns=["format"]).sort_values(by=col_program).reset_index(drop=True)
    df_online_bachelor_programs = df_online_programs[df_online_programs['level'] == 'bachelor'].drop(columns=["format"]).sort_values(by=col_program).reset_index(drop=True)
    print("База программ обработана")
except:
    print("Потерялся " + programs_file + " - нужна база программ")


try:# Шаблон дашборда / template
    print("Начинаем считывать шаблон дашборда")
    # создаем дашборд по магистратурам добавляя туда программы из базы
    df_master_dashboard = pd.read_excel(relative_folder + template_file)
    df_master_dashboard = pd.concat([df_online_master_programs, df_master_dashboard])
    df_master_dashboard['program_bitrix'] = df_master_dashboard['program_bitrix'].fillna("")
    df_master_dashboard = df_master_dashboard.fillna(0)
    df_master_dashboard[['plan_rus', 'plan_foreign']] = df_master_dashboard[['plan_rus', 'plan_foreign']].astype(int)
    print("Шаблон дашборда считан")
except:
    print("Потерялся " + template_file + " - без него дашборд не собрать")


# df_bachelor_dashboard = pd.read_excel(relative_folder + template_file)

try:# Число лидов. Почему-то это html таблица, хотя файл xls
    print("Начинаем считывать данные от Битрикса")
    df_bitrix = pd.read_html(relative_folder + bitrix_file, header=0)[0]
    df_bitrix['Образовательная программа'].fillna(main_studyonline, inplace=True)
    print("Данные от Битрикса считаны")
    # pd.read_excel(relative_folder + bitrix_file)
except:
    print("Нет выгрузки из Битрикса или она называется не " + bitrix_file)
    df_bitrix = pd.DataFrame()

leads = df_bitrix.groupby('Образовательная программа')['Образовательная программа'].count()
leads = leads * 20 #only for test! REMOVE LATER
leads = pd.DataFrame({'program_bitrix':leads.index, 'values':leads.values})
# leads[col_program] = insert_values()
df_master_dashboard[col_leads] = insert_values(df_master_dashboard, leads, 'program_bitrix', col_leads)
main_leads = leads.loc[leads['program_bitrix'] == main_studyonline, 'values'].values[0]

# АСАВ
try:
    print("Начинаем считывать данные от АСАВ")
    df_master = pd.read_excel(relative_folder + master_file, sheet_name=master_file_sheet_name, skiprows=1, usecols="L:DT")
    print("Данные от АСАВ считаны")
except:
    print("Ошибка в обработке АСАВ, возможно нет выгрузки из АСАВ или она называется не " + master_file)
    df_master = pd.DataFrame()


In [ ]:
# достаем данные по ЛК, договорам, оплатам и зачислениям из АСАВ
master_applications = df_master.groupby(master_col_programs)[master_col_programs].count() #.rename("program")#.sort_values(ascending=False)
master_applications = pd.DataFrame({col_program:master_applications.index, 'values':master_applications.values})
df_master_dashboard[col_applications] = insert_values(df_master_dashboard, master_applications, col_program, col_applications)

master_contracts = df_master[df_master[master_col_contracts].notna()].groupby(master_col_programs)[master_col_programs].count()
master_contracts = pd.DataFrame({col_program:master_contracts.index, 'values':master_contracts.values})
df_master_dashboard[col_contracts] = insert_values(df_master_dashboard, master_contracts, col_program, col_contracts)

master_payments = df_master[df_master[master_col_payments] == "Оплачено"].groupby(master_col_programs)[master_col_programs].count()
master_payments = pd.DataFrame({col_program:master_payments.index, 'values':master_payments.values})
df_master_dashboard[col_payments] = insert_values(df_master_dashboard, master_payments, col_program, col_payments)

master_enrollments = df_master[df_master[master_col_enrollments].notna()].groupby(master_col_programs)[master_col_programs].count()
master_enrollments = pd.DataFrame({col_program:master_enrollments.index, 'values':master_enrollments.values})
df_master_dashboard[col_enrollments] = insert_values(df_master_dashboard, master_enrollments, col_program, col_enrollments)

In [ ]:
df_master_dashboard[col_leads_total]                            = df_master_dashboard[col_leads_partners] + df_master_dashboard[col_leads]
df_master_dashboard[col_conversion_leads_to_contracts]          = df_master_dashboard[col_contracts] / df_master_dashboard[col_leads_total]
df_master_dashboard[col_needed_applications]                    = round(df_master_dashboard[col_plan]/ NEEDED_APPLICATIONS_RATIO)
df_master_dashboard[col_conversion_applications_to_contracts]   = df_master_dashboard[col_contracts] / df_master_dashboard[col_applications]
df_master_dashboard[col_conversion_contracts_to_payments]       = df_master_dashboard[col_payments]  / df_master_dashboard[col_contracts]
df_master_dashboard[col_conversion_contracts_to_enrollments]    = df_master_dashboard[col_payments]  / df_master_dashboard[col_contracts]
df_master_dashboard[col_payments_div_plan]                      = df_master_dashboard[col_payments]  / df_master_dashboard[col_plan]

In [ ]:
%pip install itables
from itables import show
show(df_master_dashboard.drop(columns=['program_bitrix']))

In [ ]:
df_main_dashboard


In [ ]:
df_main_dashboard = pd.DataFrame(columns=df_master_dashboard.columns)
df_main_dashboard

In [ ]:
df_main_dashboard.loc[len(df_main_dashboard)] = {col_program: main_studyonline, col_leads: main_leads}


In [ ]:
df = pd.concat([df_main_dashboard, df_master_dashboard], sort=False, reset)
df

In [ ]:
df_master_dashboard.style.bar(subset=[col_payments_div_plan], caolor='#d65f5f')

In [ ]:


#df_master_dashboard.drop(columns=['program_bitrix']).to_excel(relative_folder + dashboard_file, float_format="%.2f", index=False)
# try:
#     df_bachelor_app = pd.read_excel(relative_folder + bachelor_app_file)
# except:
#     print("Нет выгрузки заявлений из АИС ПК или она называется не " + bachelor_app_file)
#     df_bachelor_app = pd.DataFrame()

# try:
#     df_bachelor_con = pd.read_excel(relative_folder + bachelor_con_file)
# except:
#     print("Нет выгрузки договоров из АИС ПК или она называется не " + bachelor_con_file)
#     df_bachelor_con = pd.DataFrame()

# try:
#     df_bachelor_enr = pd.read_excel(relative_folder + bachelor_enr_file)
# except:
#     print("Нет выгрузки зачислений из АИС ПК или она называется не " + bachelor_enr_file)
#     df_bachelor_enr = pd.DataFrame()

# df_master_dashboard[col_applications] =
# print(df_master.info())
# print(df_master.groupby(master_col_programs)[master_col_programs].count()) #.reset_index())

# df = pd.concat([df_master_dashboard, df_bachelor_dashboard, df_main_dashboard])



## Stage 2.5

In [13]:
import pandas as pd
pd.set_option('display.max_columns', None)
# папки и файлы для загрузки
relative_folder = "../data/"

programs_file = "programs.xlsx"

dashboard_file = "dashboard.xlsx"

bitrix_file = "bitrix.xls"

master_file = "asav.xlsx"
master_file_sheet_name = "только онлайн"
master_col_programs = "Конкурс в магистратуру"
master_col_contracts = "Договор на обучение" # не пусто
master_col_payments = "Оплата первого периода" # "Оплачено"
master_col_enrollments = "Приказ о зачислении" # не пусто


bachelor_app_file = "bac_applications.xls"

bachelor_con_file = "bac_contracts.xls"

bachelor_enr_file = "bac_enrolled.xlsx"

bachelor_col_programs = "Конкурсная группа"
bachelor_col_programs_names = "Образовательная программа"
# bachelor_col_contracts = "Образовательная программа"
bachelor_col_payments = "Статус оплаты" # "Оплачен" или "Оплачен по квитанциям"
bachelor_col_enrollments = "Конкурс"

main_studyonline = "Общий лендинг"

template_file = "template.xlsx"

# основные параметры
col_program = 'program'
col_plan = 'plan_rus'
col_leads = "Общее кол-во заявок (studyonline) ВСЕГО"
col_applications = """Кол-во регистраций в ЛК абитуриента (РФ 1 и 2 приоритет)"""
col_contracts = """Договоры (ПК) РФ"""
col_payments = """Оплаты (ПК)"""
col_enrollments = "Зачисленные (ПК)"


# дополнительные и расчетные параметры
col_leads_total = """Общее кол-во заявок ВСЕГО (портал+РК)"""
col_leads_partners = """Общее кол-во заявок  c  hse.ru по кнопкам/партнерских стр"""
col_conversion_leads_to_contracts = """Конверсия заявка -> договор (без учета заявок с общего ленда)"""
col_needed_applications = """Необходимо регистраций в ЛК для обеспечения набора (30% поступили от регистрации в АСАВ)"""
NEEDED_APPLICATIONS_RATIO = 30 / 100 #percents
col_conversion_applications_to_contracts = """Конверсия ЛК -> договор"""
col_conversion_contracts_to_payments = """Конверсия договор -> оплата"""
col_conversion_contracts_to_enrollments = """Конверсия договор -> зачисление"""
col_payments_div_plan = """Выполнение плана %"""
col_income_1year     = "Выручка за 1 год"
col_income_all       = "Выручка за весь период обучения"
col_income_1year_hse = "Выручка за 1 год после отчислений партнерам"
col_income_all_hse   = "Выручка за весь период обучения после отчислений партнерам"

# считывание файлов

try:
    # cчитываем базу данных програм
    print("Начинаем считывать базу программ")
    df_online_programs = pd.read_excel(relative_folder + programs_file)
    df_online_programs = df_online_programs[df_online_programs['format'] != 'offline'].reset_index(drop=True)
    #df_online_programs[['plan_rus', 'plan_foreign']] = df_online_programs[['plan_rus', 'plan_foreign']].astype(int)
    df_online_master_programs = df_online_programs[df_online_programs['level'] == 'master'].drop(columns=["format"]).sort_values(by=col_program).reset_index(drop=True)
    df_online_bachelor_programs = df_online_programs[df_online_programs['level'] == 'bachelor'].drop(columns=["format"]).sort_values(by=col_program).reset_index(drop=True)
    print("База программ обработана")
except:
    print("!!!Потерялся " + programs_file + " - нужна база программ")
    #return "Error program database"


try:# Шаблон дашборда / template
    print("Начинаем считывать шаблон дашборда")
    # создаем дашборд по магистратурам добавляя туда программы из базы
    df_master_dashboard = pd.read_excel(relative_folder + template_file)
    df_master_dashboard = pd.concat([df_online_master_programs, df_master_dashboard])
    df_master_dashboard['program_bitrix'] = df_master_dashboard['program_bitrix'].fillna("")
    df_master_dashboard = df_master_dashboard.fillna(0)
    df_master_dashboard[['plan_rus', 'plan_foreign']] = df_master_dashboard[['plan_rus', 'plan_foreign']].astype(int)

    df_bachelor_dashboard = pd.read_excel(relative_folder + template_file)
    df_bachelor_dashboard = pd.concat([df_online_bachelor_programs, df_bachelor_dashboard])
    df_bachelor_dashboard['program_bitrix'] = df_bachelor_dashboard['program_bitrix'].fillna("")
    df_bachelor_dashboard = df_bachelor_dashboard.fillna(0)
    df_bachelor_dashboard[['plan_rus', 'plan_foreign']] = df_bachelor_dashboard[['plan_rus', 'plan_foreign']].astype(int)
    print("Шаблон дашборда считан")
except:
    print("!!!Потерялся " + template_file + " - без него дашборд не собрать")
    #return "Error dashboard template"

try:# Число лидов. Почему-то это html таблица, хотя файл xls
    print("Начинаем считывать данные от Битрикса")
    df_bitrix = pd.read_html(relative_folder + bitrix_file, header=0)[0]
    df_bitrix['Образовательная программа'].fillna(main_studyonline, inplace=True)
    print("Данные от Битрикса считаны")
    # pd.read_excel(relative_folder + bitrix_file)
except:
    print("Нет выгрузки из Битрикса или она называется не " + bitrix_file)
    df_bitrix = pd.DataFrame()

try:
    leads = df_bitrix.groupby('Образовательная программа')['Образовательная программа'].count()
    leads = leads * 20 #only for test! REMOVE LATER
    leads = pd.DataFrame({'program_bitrix':leads.index, 'values':leads.values})
except:
    leads = pd.DataFrame(columns=['program_bitrix', 'values'])
df_master_dashboard[col_leads]   = insert_values(df_master_dashboard,   leads, 'program_bitrix', col_leads)
df_bachelor_dashboard[col_leads] = insert_values(df_bachelor_dashboard, leads, 'program_bitrix', col_leads)
main_leads = leads.loc[leads['program_bitrix'] == main_studyonline, 'values'].values[0]

# АСАВ
try:
    print("Начинаем считывать данные от АСАВ")
    df_master = pd.read_excel(relative_folder + master_file, sheet_name=master_file_sheet_name, skiprows=1, usecols="L:DT")
    print("Данные от АСАВ считаны")
except:
    print("Ошибка в обработке АСАВ, возможно нет выгрузки из АСАВ или она называется не " + master_file)
    df_master = pd.DataFrame(columns=[master_col_programs, master_col_contracts, master_col_payments, master_col_enrollments])

# достаем данные по ЛК, договорам, оплатам и зачислениям из АСАВ
master_applications = df_master.groupby(master_col_programs)[master_col_programs].count() #.rename("program")#.sort_values(ascending=False)
master_applications = pd.DataFrame({col_program:master_applications.index, 'values':master_applications.values})
df_master_dashboard[col_applications] = insert_values(df_master_dashboard, master_applications, col_program, col_applications)

master_contracts = df_master[df_master[master_col_contracts].notna()].groupby(master_col_programs)[master_col_programs].count()
master_contracts = pd.DataFrame({col_program:master_contracts.index, 'values':master_contracts.values})
df_master_dashboard[col_contracts] = insert_values(df_master_dashboard, master_contracts, col_program, col_contracts)

master_payments = df_master[df_master[master_col_payments] == "Оплачено"].groupby(master_col_programs)[master_col_programs].count()
master_payments = pd.DataFrame({col_program:master_payments.index, 'values':master_payments.values})
df_master_dashboard[col_payments] = insert_values(df_master_dashboard, master_payments, col_program, col_payments)

master_enrollments = df_master[df_master[master_col_enrollments].notna()].groupby(master_col_programs)[master_col_programs].count()
master_enrollments = pd.DataFrame({col_program:master_enrollments.index, 'values':master_enrollments.values})
df_master_dashboard[col_enrollments] = insert_values(df_master_dashboard, master_enrollments, col_program, col_enrollments)



# АИС ПК
try:
    print("Начинаем считывать данные от АИС ПК")
    df_bachelor_app = pd.read_excel(relative_folder + bachelor_app_file, usecols="I:Z") #, sheet_name=master_file_sheet_name, skiprows=1, usecols="L:DT")
    df_bachelor_con = pd.read_excel(relative_folder + bachelor_con_file, usecols="H:T") #, sheet_name=master_file_sheet_name, skiprows=1, usecols="L:DT")
    df_bachelor_enr = pd.read_excel(relative_folder + bachelor_enr_file, usecols="E:H") #, sheet_name=master_file_sheet_name, skiprows=1)
    print("Данные от АИС ПК считаны")

    bachelor_dict = {
'Глобальные цифровые коммуникации (онлайн)'                        :'Глобальные цифровые коммуникации - онлайн (О К)'        ,
'Глобальные цифровые коммуникации (онлайн)'                        :'Глобальные цифровые коммуникации (Медиа) - онлайн (О К)',
'Компьютерные науки и анализ данных (онлайн)'                      :'Компьютерные науки и анализ данных - онлайн (О К)'      ,
'Экономический анализ (онлайн)'                                    :'Экономический анализ - онлайн (О К)'                    ,
'Дизайн  (онлайн)'                                                 :'Дизайн - онлайн (О К)'                                  ,
'Программные системы и автоматизация процессов разработки (онлайн)':'Программные системы и автоматизация процессов разработки - онлайн (О К)'
    }
except:
    print("Ошибка в обработке АИС ПК, возможно нет выгрузки из АИС ПК или она называется не:\n" + bachelor_app_file)
    print(bachelor_con_file)
    print(bachelor_enr_file)
    # df_master = pd.DataFrame(columns=[master_col_programs, master_col_contracts, master_col_payments, master_col_enrollments])

# достаем данные по ЛК, договорам, оплатам и зачислениям из АСАВ
bachelor_applications = df_bachelor_app.groupby(bachelor_col_programs)[bachelor_col_programs].count() #.rename("program")#.sort_values(ascending=False)
bachelor_applications = pd.DataFrame({col_program:bachelor_applications.index, 'values':bachelor_applications.values})
df_bachelor_dashboard[col_applications] = insert_values(df_bachelor_dashboard, bachelor_applications, col_program, col_applications)

bachelor_contracts = df_bachelor_con.groupby(bachelor_col_programs_names)[bachelor_col_programs_names].count()
bachelor_contracts = bachelor_contracts.rename(index=bachelor_dict)
bachelor_contracts = pd.DataFrame({col_program:bachelor_contracts.index, 'values':bachelor_contracts.values})
df_bachelor_dashboard[col_contracts] = insert_values(df_bachelor_dashboard, bachelor_contracts, col_program, col_contracts)

bachelor_payments = df_bachelor_con[(df_bachelor_con[bachelor_col_payments] == "Оплачен")|(df_bachelor_con[bachelor_col_payments] == "Оплачен по квитанциям")].groupby(bachelor_col_programs_names)[bachelor_col_programs_names].count()
bachelor_payments = bachelor_payments.rename(index=bachelor_dict)
bachelor_payments = pd.DataFrame({col_program:bachelor_payments.index, 'values':bachelor_payments.values})
df_bachelor_dashboard[col_payments] = insert_values(df_bachelor_dashboard, bachelor_payments, col_program, col_payments)

bachelor_enrollments = df_bachelor_enr.groupby(bachelor_col_enrollments)[bachelor_col_enrollments].count()
bachelor_enrollments = pd.DataFrame({col_program:bachelor_enrollments.index, 'values':bachelor_enrollments.values})
df_bachelor_dashboard[col_enrollments] = insert_values(df_bachelor_dashboard, bachelor_enrollments, col_program, col_enrollments)


df_main_dashboard = pd.DataFrame(columns=df_master_dashboard.columns)
df_main_dashboard.loc[len(df_main_dashboard)] = {col_program: main_studyonline, col_leads: main_leads}
df = pd.concat([df_main_dashboard, df_master_dashboard, df_bachelor_dashboard], ignore_index=True, sort=False).drop(columns=['program_bitrix'])


# считаем второстепенные столбцы
df[col_leads_total]                            = df[col_leads_partners] + df[col_leads]
df[col_conversion_leads_to_contracts]          = df[col_contracts] / df[col_leads_total]
df[col_needed_applications]              = round(df[col_plan]/ NEEDED_APPLICATIONS_RATIO)
df[col_conversion_applications_to_contracts]   = df[col_contracts] / df[col_applications]
df[col_conversion_contracts_to_payments]       = df[col_payments]  / df[col_contracts]
df[col_conversion_contracts_to_enrollments]    = df[col_payments]  / df[col_contracts]
df[col_payments_div_plan]                      = df[col_payments]  / df[col_plan]
df[col_income_1year     ] = df['price'] * df[col_payments]
# df[col_income_all       ] = df['price'] * df[col_payments] * (2 if df['level'] == 'master' else 4)
# df[col_income_1year_hse ] = df[col_income_1year] * df['income_percent'] / 100
# df[col_income_all_hse   ] = df[col_income_all] * df['income_percent'] / 100

df.to_excel("dashboard.xlsx")

Начинаем считывать базу программ
База программ обработана
Начинаем считывать шаблон дашборда
Шаблон дашборда считан
Начинаем считывать данные от Битрикса
Данные от Битрикса считаны
Начинаем считывать данные от АСАВ
Данные от АСАВ считаны
Начинаем считывать данные от АИС ПК
Данные от АИС ПК считаны


## Stage 3

In [ ]:
%pip install --upgrade google-api-python-client

In [ ]:
%pip install gspread oauth2client

In [ ]:
%cat ../credentials.json

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Define the scope and authenticate
scope = ["https://www.googleapis.com/auth/spreadsheets", "https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("../credentials.json", scope)
client = gspread.authorize(creds)

# Open Google Spreadsheet
spreadsheet = client.open("Еженедельный отчет 2025_общий")
worksheet = spreadsheet.dashboard

# Clear the worksheet before updating
worksheet.clear()

# Update the worksheet with the aggregated data
worksheet.update([df.columns.values.tolist()] + df.values.tolist())

print("Google Spreadsheet updated successfully.")

In [ ]:
# Copyright 2018 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# [START sheets_quickstart]
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/spreadsheets.readonly"]

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = "1BxiMVs0XRA5nFMdKvBdBZjgmUUqptlbs74OgvE2upms"
SAMPLE_RANGE_NAME = "Class Data!A2:E"


def main():
  """Shows basic usage of the Sheets API.
  Prints values from a sample spreadsheet.
  """
  creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
  if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "../credentials.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())

  try:
    service = build("sheets", "v4", credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result = (
        sheet.values()
        .get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME)
        .execute()
    )
    values = result.get("values", [])

    if not values:
      print("No data found.")
      return

    print("Name, Major:")
    for row in values:
      # Print columns A and E, which correspond to indices 0 and 4.
      print(f"{row[0]}, {row[4]}")
  except HttpError as err:
    print(err)


if __name__ == "__main__":
  main()
# [END sheets_quickstart]

# Stage 4

In [19]:
%pip install numpy pandas openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.


In [20]:
import pandas as pd

def insert_values(df_dashboard, df_values, col_join, col_values): # df_values should have "values" column
    df_dashboard.loc[df_dashboard[col_join].isin(df_values[col_join]), col_values] = df_values.loc[df_values[col_join].isin(df_dashboard[col_join]), 'values'].values
    df_dashboard[col_values] = df_dashboard[col_values].fillna(0).astype(int)
    return df_dashboard[col_values]

def process_excel_files():
    import numpy as np

    # папки и файлы для загрузки
    relative_folder = "../data/"

    programs_file = "programs.xlsx"

    dashboard_file = "dashboard.xlsx"

    bitrix_file = "bitrix.xls"

    master_file = "asav.xlsx"
    master_file_sheet_name = "только онлайн"
    master_col_programs = "Конкурс в магистратуру"
    master_col_contracts = "Договор на обучение" # не пусто
    master_col_payments = "Оплата первого периода" # "Оплачено"
    master_col_enrollments = "Приказ о зачислении" # не пусто


    bachelor_app_file = "bac_applications.xls"

    bachelor_con_file = "bac_contracts.xls"

    bachelor_enr_file = "bac_enrolled.xlsx"

    bachelor_col_programs = "Конкурсная группа"
    bachelor_col_programs_names = "Образовательная программа"
    # bachelor_col_contracts = "Образовательная программа"
    bachelor_col_payments = "Статус оплаты" # "Оплачен" или "Оплачен по квитанциям"
    bachelor_col_enrollments = "Конкурс"

    main_studyonline = "Общий лендинг"

    template_file = "template.xlsx"


    # основные параметры
    col_program = 'program'
    col_plan = 'plan_rus'
    col_leads = "Общее кол-во заявок (studyonline) ВСЕГО"
    col_applications = """Кол-во регистраций в ЛК абитуриента (РФ 1 и 2 приоритет)"""
    col_contracts = """Договоры (ПК) РФ"""
    col_payments = """Оплаты (ПК)"""
    col_enrollments = "Зачисленные (ПК)"


    # дополнительные и расчетные параметры
    col_leads_total = """Общее кол-во заявок ВСЕГО (портал+РК)"""
    col_leads_partners = """Общее кол-во заявок  c  hse.ru по кнопкам/партнерских стр"""
    col_conversion_leads_to_contracts = """Конверсия заявка -> договор (без учета заявок с общего ленда)"""
    col_needed_applications = """Необходимо регистраций в ЛК для обеспечения набора (30% поступили от регистрации в АСАВ)"""
    NEEDED_APPLICATIONS_RATIO = 30 / 100 #percents
    col_conversion_applications_to_contracts = """Конверсия ЛК -> договор"""
    col_conversion_contracts_to_payments = """Конверсия договор -> оплата"""
    col_conversion_contracts_to_enrollments = """Конверсия договор -> зачисление"""
    col_payments_div_plan = """Выполнение плана %"""
    col_income_1year     = "Выручка за 1 год"
    col_income_all       = "Выручка за весь период обучения"
    col_income_1year_hse = "Выручка за 1 год после отчислений партнерам"
    col_income_all_hse   = "Выручка за весь период обучения после отчислений партнерам"


    # считывание файлов

    try:
        # cчитываем базу данных програм
        print("Начинаем считывать базу программ")
        df_online_programs = pd.read_excel(relative_folder + programs_file)
        df_online_programs = df_online_programs[df_online_programs['format'] != 'offline'].reset_index(drop=True)
        #df_online_programs[['plan_rus', 'plan_foreign']] = df_online_programs[['plan_rus', 'plan_foreign']].astype(int)
        df_online_master_programs = df_online_programs[df_online_programs['level'] == 'master'].drop(columns=["format"]).sort_values(by=col_program).reset_index(drop=True)
        df_online_bachelor_programs = df_online_programs[df_online_programs['level'] == 'bachelor'].drop(columns=["format"]).sort_values(by=col_program).reset_index(drop=True)
        print("База программ обработана")
    except:
        print("Потерялся " + programs_file + " - нужна база программ")
        return "Error program database"


    try:# Шаблон дашборда / template
        print("Начинаем считывать шаблон дашборда")
        # создаем дашборд по магистратурам добавляя туда программы из базы
        df_master_dashboard = pd.read_excel(relative_folder + template_file)
        df_master_dashboard = pd.concat([df_online_master_programs, df_master_dashboard])
        df_master_dashboard['program_bitrix'] = df_master_dashboard['program_bitrix'].fillna("")
        df_master_dashboard = df_master_dashboard.fillna(0)
        df_master_dashboard[['plan_rus', 'plan_foreign']] = df_master_dashboard[['plan_rus', 'plan_foreign']].astype(int)

        df_bachelor_dashboard = pd.read_excel(relative_folder + template_file)
        df_bachelor_dashboard = pd.concat([df_online_bachelor_programs, df_bachelor_dashboard])
        df_bachelor_dashboard['program_bitrix'] = df_bachelor_dashboard['program_bitrix'].fillna("")
        df_bachelor_dashboard = df_bachelor_dashboard.fillna(0)
        df_bachelor_dashboard[['plan_rus', 'plan_foreign']] = df_bachelor_dashboard[['plan_rus', 'plan_foreign']].astype(int)
        print("Шаблон дашборда считан")
    except:
        print("Потерялся " + template_file + " - без него дашборд не собрать")
        return "Error dashboard template"

    try:# Число лидов. Почему-то это html таблица, хотя файл xls
        print("Начинаем считывать данные от Битрикса")
        df_bitrix = pd.read_html(relative_folder + bitrix_file, header=0)[0]
        df_bitrix['Образовательная программа'].fillna(main_studyonline, inplace=True)
        print("Данные от Битрикса считаны")
        # pd.read_excel(relative_folder + bitrix_file)
    except:
        print("Нет выгрузки из Битрикса или она называется не " + bitrix_file)
        df_bitrix = pd.DataFrame()

    try:
        leads = df_bitrix.groupby('Образовательная программа')['Образовательная программа'].count()
        leads = leads * 20 #only for test! REMOVE LATER
        leads = pd.DataFrame({'program_bitrix':leads.index, 'values':leads.values})
    except:
        leads = pd.DataFrame(columns=['program_bitrix', 'values'])
    df_master_dashboard[col_leads] = insert_values(df_master_dashboard, leads, 'program_bitrix', col_leads)
    main_leads = leads.loc[leads['program_bitrix'] == main_studyonline, 'values'].values[0]

    # АСАВ
    try:
        print("Начинаем считывать данные от АСАВ")
        df_master = pd.read_excel(relative_folder + master_file, sheet_name=master_file_sheet_name, skiprows=1, usecols="L:DT")
        print("Данные от АСАВ считаны")
    except:
        print("Ошибка в обработке АСАВ, возможно нет выгрузки из АСАВ или она называется не " + master_file)
        df_master = pd.DataFrame(columns=[master_col_programs, master_col_contracts, master_col_payments, master_col_enrollments])

    # достаем данные по ЛК, договорам, оплатам и зачислениям из АСАВ
    master_applications = df_master.groupby(master_col_programs)[master_col_programs].count() #.rename("program")#.sort_values(ascending=False)
    master_applications = pd.DataFrame({col_program:master_applications.index, 'values':master_applications.values})
    df_master_dashboard[col_applications] = insert_values(df_master_dashboard, master_applications, col_program, col_applications)

    master_contracts = df_master[df_master[master_col_contracts].notna()].groupby(master_col_programs)[master_col_programs].count()
    master_contracts = pd.DataFrame({col_program:master_contracts.index, 'values':master_contracts.values})
    df_master_dashboard[col_contracts] = insert_values(df_master_dashboard, master_contracts, col_program, col_contracts)

    master_payments = df_master[df_master[master_col_payments] == "Оплачено"].groupby(master_col_programs)[master_col_programs].count()
    master_payments = pd.DataFrame({col_program:master_payments.index, 'values':master_payments.values})
    df_master_dashboard[col_payments] = insert_values(df_master_dashboard, master_payments, col_program, col_payments)

    master_enrollments = df_master[df_master[master_col_enrollments].notna()].groupby(master_col_programs)[master_col_programs].count()
    master_enrollments = pd.DataFrame({col_program:master_enrollments.index, 'values':master_enrollments.values})
    df_master_dashboard[col_enrollments] = insert_values(df_master_dashboard, master_enrollments, col_program, col_enrollments)




    # АИС ПК
    try:
        print("Начинаем считывать данные от АИС ПК")
        df_bachelor_app = pd.read_excel(relative_folder + bachelor_app_file, usecols="I:Z") #, sheet_name=master_file_sheet_name, skiprows=1, usecols="L:DT")
        df_bachelor_con = pd.read_excel(relative_folder + bachelor_con_file, usecols="H:T") #, sheet_name=master_file_sheet_name, skiprows=1, usecols="L:DT")
        df_bachelor_enr = pd.read_excel(relative_folder + bachelor_enr_file, usecols="E:H") #, sheet_name=master_file_sheet_name, skiprows=1)
        print("Данные от АИС ПК считаны")

        bachelor_dict = {
    'Глобальные цифровые коммуникации (онлайн)'                        :'Глобальные цифровые коммуникации - онлайн (О К)'        ,
    'Глобальные цифровые коммуникации (онлайн)'                        :'Глобальные цифровые коммуникации (Медиа) - онлайн (О К)',
    'Компьютерные науки и анализ данных (онлайн)'                      :'Компьютерные науки и анализ данных - онлайн (О К)'      ,
    'Экономический анализ (онлайн)'                                    :'Экономический анализ - онлайн (О К)'                    ,
    'Дизайн  (онлайн)'                                                 :'Дизайн - онлайн (О К)'                                  ,
    'Программные системы и автоматизация процессов разработки (онлайн)':'Программные системы и автоматизация процессов разработки - онлайн (О К)'
        }
    except:
        print("Ошибка в обработке АИС ПК, возможно нет выгрузки из АИС ПК или она называется не:\n" + bachelor_app_file)
        print(bachelor_con_file)
        print(bachelor_enr_file)
        # df_master = pd.DataFrame(columns=[master_col_programs, master_col_contracts, master_col_payments, master_col_enrollments])

    # достаем данные по ЛК, договорам, оплатам и зачислениям из АСАВ
    bachelor_applications = df_bachelor_app.groupby(bachelor_col_programs)[bachelor_col_programs].count() #.rename("program")#.sort_values(ascending=False)
    bachelor_applications = pd.DataFrame({col_program:bachelor_applications.index, 'values':bachelor_applications.values})
    df_bachelor_dashboard[col_applications] = insert_values(df_bachelor_dashboard, bachelor_applications, col_program, col_applications)

    bachelor_contracts = df_bachelor_con.groupby(bachelor_col_programs_names)[bachelor_col_programs_names].count()
    bachelor_contracts = bachelor_contracts.rename(index=bachelor_dict)
    bachelor_contracts = pd.DataFrame({col_program:bachelor_contracts.index, 'values':bachelor_contracts.values})
    df_bachelor_dashboard[col_contracts] = insert_values(df_bachelor_dashboard, bachelor_contracts, col_program, col_contracts)

    bachelor_payments = df_bachelor_con[(df_bachelor_con[bachelor_col_payments] == "Оплачен")|(df_bachelor_con[bachelor_col_payments] == "Оплачен по квитанциям")].groupby(bachelor_col_programs_names)[bachelor_col_programs_names].count()
    bachelor_payments = bachelor_payments.rename(index=bachelor_dict)
    bachelor_payments = pd.DataFrame({col_program:bachelor_payments.index, 'values':bachelor_payments.values})
    df_bachelor_dashboard[col_payments] = insert_values(df_bachelor_dashboard, bachelor_payments, col_program, col_payments)

    bachelor_enrollments = df_bachelor_enr.groupby(bachelor_col_enrollments)[bachelor_col_enrollments].count()
    bachelor_enrollments = pd.DataFrame({col_program:bachelor_enrollments.index, 'values':bachelor_enrollments.values})
    df_bachelor_dashboard[col_enrollments] = insert_values(df_bachelor_dashboard, bachelor_enrollments, col_program, col_enrollments)


    df_main_dashboard = pd.DataFrame(columns=df_master_dashboard.columns)
    df_main_dashboard.loc[len(df_main_dashboard)] = {col_program: main_studyonline, col_leads: main_leads}
    df = pd.concat([df_main_dashboard, df_master_dashboard, df_bachelor_dashboard], ignore_index=True, sort=False).drop(columns=['program_bitrix'])


    # считаем второстепенные столбцы
    df[col_leads_total]                            = df[col_leads_partners] + df[col_leads]
    df[col_conversion_leads_to_contracts]          = df[col_contracts] / df[col_leads_total]
    df[col_needed_applications]              = round(df[col_plan]/ NEEDED_APPLICATIONS_RATIO)
    df[col_conversion_applications_to_contracts]   = df[col_contracts] / df[col_applications]
    df[col_conversion_contracts_to_payments]       = df[col_payments]  / df[col_contracts]
    df[col_conversion_contracts_to_enrollments]    = df[col_payments]  / df[col_contracts]
    df[col_payments_div_plan]                      = df[col_payments]  / df[col_plan]
    df[col_income_1year     ] = df['price'] * df[col_payments]
    # df[col_income_all       ] = df['price'] * df[col_payments] * (2 if df['level'] == 'master' else 4)
    # df[col_income_1year_hse ] = df[col_income_1year] * df['income_percent'] / 100
    # df[col_income_all_hse   ] = df[col_income_all] * df['income_percent'] / 100

    df_main_dashboard = pd.DataFrame(columns=df_master_dashboard.columns)
    df_main_dashboard.loc[len(df_main_dashboard)] = {col_program: main_studyonline, col_leads: main_leads}
    df = pd.concat([df_main_dashboard, df_master_dashboard], ignore_index=True, sort=False) #df_bachelor_dashboard,

    df.replace(np.inf, 0, inplace=True)

    return df

In [23]:
df = process_excel_files()

Начинаем считывать базу программ
База программ обработана
Начинаем считывать шаблон дашборда
Шаблон дашборда считан
Начинаем считывать данные от Битрикса
Нет выгрузки из Битрикса или она называется не bitrix.xls


/var/folders/vq/48g1pl3n2n58zsl0snh1wnf00000gn/T/ipykernel_18434/4118933506.py:93: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_master_dashboard = df_master_dashboard.fillna(0)
/var/folders/vq/48g1pl3n2n58zsl0snh1wnf00000gn/T/ipykernel_18434/4118933506.py:99: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_bachelor_dashboard = df_bachelor_dashboard.fillna(0)
/var/folders/vq/48g1pl3n2n58zsl0snh1wnf00000gn/T/ipykernel_18434/4118933506.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version o

IndexError: index 0 is out of bounds for axis 0 with size 0

In [4]:
display(df)

NameError: name 'df' is not defined